In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import convolve2d
from skimage.io import imread

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
rootfolder = ".."

Define the function to compute the kernel given the weights and the degree of the polynomial


In [ ]:
def compute_2D_LPA_kernel(w, N):
    """
    Compute the 2D LPA kernel for a given weights and polynomial degree

    Input:
        w: matrix containing the weights for the local LS problem
        N: degree of the polynomial approximation
    Return:
        g: the computed LPA kernel
    """
    # window size is the lenght of the weight vector
    r, c = w.shape
    M = r * c

    # create the matrix T
    tx = np.linspace(0, 1, c)
    ty = np.linspace(0, 1, r)
    tx, ty = np.meshgrid(tx, ty)
    tx = tx.reshape(-1)
    ty = ty.reshape(-1)
    T = np.zeros((M, (N + 1) ** 2))
    cnt = 0
    for i in range(N + 1):
        for j in range(N - i + 1):
            if i == 0 and j == 0:
                T[:, cnt] = np.ones(M)
            else:
                T[:, cnt] = tx**i * ty**j
            cnt = cnt + 1
    T = T[:, :cnt]

    # unroll the matrix of the weights
    w = w.reshape(-1)

    # generate the inverse of weights
    winv = np.zeros_like(w)
    winv[w != 0] = 1 / w[w != 0]

    # set to zero weights that are inf
    winv[np.isinf(winv)] = 0

    # define the weight matrix
    W = np.diag(w)
    Winv = np.diag(winv)

    ## construct the LPA kernel

    # compute the qr decomposition of WT
    Q, R = np.linalg.qr(W @ T)

    # define Qtilde
    Qtilde = Winv @ Q

    # adjust Qtilde with the weights matrix squared
    W2Qtilde = W @ W @ Qtilde

    # select the central row of W2Qtilde
    row = M // 2

    # compute the kernel
    g_bar = W2Qtilde[row, 0] * Qtilde[:, 0]

    # reshape the kernel in a matrix
    g_bar = g_bar.reshape(r, c)

    # flipping, since it is used in convolution
    g = np.flip(g_bar)

    return g

Load the image and add the noise


In [ ]:
img = imread(f"{rootfolder}/data/cameraman.png") / 255

sigma_noise = 20 / 255
noisy_img = img + np.random.normal(size=img.shape) * sigma_noise

psnr_noisy = 10 * np.log10(1 / np.mean((noisy_img - img) ** 2))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img, cmap="gray")
ax[0].set_title("Original image")

ax[1].imshow(noisy_img, cmap="gray")
ax[1].set_title(f"Noisy image, PSNR = {psnr_noisy:.2f}")

## LPA-ICI 2D

Set the LPA-ICI parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 1

# parameter for the confidence intervals in the ICI rule
Gamma = 2

# Set all the scale values
hmax = 21
all_h = np.arange(1, hmax + 1)

Generate the LPA kernels for all the scales. Use centered weights.


In [ ]:
all_g = []
for i, h in enumerate(all_h):
    # define the weights for the scale h symmetric
    # size of the weight MATRIX
    w = np.zeros((2 * hmax + 1, 2 * hmax + 1))
    w[hmax - h : hmax + h + 1, hmax - h : hmax + h + 1] = 1
    # compute and store the kernel g
    g = compute_2D_LPA_kernel(w, N)
    all_g.append(g)

Initialize all the variables for the ICI rule


In [ ]:
# initialize the estimate for each scale
yhat = np.zeros((img.shape))

# initialize the vector containing the best scale for each sample
best_scale = np.zeros(shape=yhat.shape)

# initialize the lower and upper bound matrices
lower_bounds = -np.inf * np.ones(shape=yhat.shape)
upper_bounds = np.inf * np.ones(shape=yhat.shape)

Loop over all the scales


In [ ]:
# Loop over all the scales
for i, h in enumerate(all_h):
    g = all_g[i]

    # compute the estimate for the scale h
    yhat_h = convolve2d(noisy_img, g, mode="same", boundary="symm")

    # compute the variance
    var_h = sigma_noise**2 * convolve2d(
        np.ones_like(noisy_img), g**2, mode="same", boundary="symm"
    )

    # compute the lower and upper bound of the confidence interval for the scale h
    lb = yhat_h - Gamma * np.sqrt(var_h)
    ub = yhat_h + Gamma * np.sqrt(var_h)

    # update the lower and upper bounds
    lower_bounds = np.maximum(lower_bounds, lb)
    upper_bounds = np.minimum(upper_bounds, ub)

    # identify for which samples h is the best scale according to the
    # ICI rule and update the best_scale vector accordingly
    valid_ici = lower_bounds <= upper_bounds
    best_scale[valid_ici] = h

    # update the estimate
    yhat[valid_ici] = yhat_h[valid_ici]


Compute the PSNR


In [ ]:
psnr = 10 * np.log10(1 / np.mean((yhat - img) ** 2))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(yhat, cmap="gray")
ax[0].set_title(f"LPA-ICI estimate, PSNR = {psnr:.2f}")

ax[1].imshow(best_scale)
ax[1].set_title("Best scale for each pixel")
fig.colorbar(ax[1].pcolormesh(best_scale), ax=ax[1])

## Anisotropic LPA-ICI

Set the parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 1

# parameter for the confidence intervals in the ICI rule
Gamma = 2

# Set all the scale values
hmax = 21
all_h = np.arange(1, hmax + 1)

# set all the direction values
all_theta = np.arange(4)

Generate the LPA kernels for all the scales and all the directions


In [ ]:
all_g = []

for theta in all_theta:
    all_g_theta = []
    for i, h in enumerate(all_h):
        # define the weights for the scale h and the direction theta
        w = np.zeros((2 * hmax + 1, 2 * hmax + 1))

        if theta == 0:  # Horizontal
            w[hmax, hmax - h : hmax + h + 1] = 1
        elif theta == 1:  # Vertical
            w[hmax - h : hmax + h + 1, hmax] = 1
        elif theta == 2:  # Diagonal 1 (top-left to bottom-right)
            for k in range(-h, h + 1):
                if 0 <= hmax + k < 2 * hmax + 1:
                    w[hmax + k, hmax + k] = 1
        elif theta == 3:  # Diagonal 2 (top-right to bottom-left)
            for k in range(-h, h + 1):
                if 0 <= hmax + k < 2 * hmax + 1 and 0 <= hmax - k < 2 * hmax + 1:
                    w[hmax + k, hmax - k] = 1

        # compute and store the kernel g
        g = compute_2D_LPA_kernel(w, N)
        all_g_theta.append(g)

    all_g.append(all_g_theta)


Initialize all the variables


In [ ]:
# initialize the estimate for each scale
yhat = np.zeros(img.shape)

# initialize the matrix of the aggregation weights
weights = np.zeros(img.shape)


Use the LPA-ICI to compute find the best scale for each direction and compute the finale estimates


In [ ]:
# loop over all the directions
for theta in all_theta:
    # initialize the estimate for the direction theta
    yhat_theta = np.zeros(img.shape)

    # initialize the matrix all the variances for the direction theta
    var_theta = np.zeros(img.shape)

    # initialize the lower and upper bounds matrices
    lower_bounds = -np.inf * np.ones(img.shape)
    upper_bounds = np.inf * np.ones(img.shape)

    # loop over all scales
    all_g_theta = all_g[theta]
    for i, h in enumerate(all_h):
        g = all_g_theta[i]

        # compute the estimate for the scale h
        yhat_h = convolve2d(noisy_img, g, mode="same", boundary="symm")

        # compute the variance
        var_h = sigma_noise**2 * convolve2d(
            np.ones_like(noisy_img), g**2, mode="same", boundary="symm"
        )

        # compute the lower and upper bound of the confidence interval for the scale h
        lb = yhat_h - Gamma * np.sqrt(var_h)
        ub = yhat_h + Gamma * np.sqrt(var_h)

        # update the lower and upper bounds
        lower_bounds = np.maximum(lower_bounds, lb)
        upper_bounds = np.minimum(upper_bounds, ub)

        # identify valid ICI points
        valid_ici = lower_bounds <= upper_bounds

        # update the estimate
        yhat_theta[valid_ici] = yhat_h[valid_ici]

        # update the matrix with the variances
        var_theta[valid_ici] = var_h[valid_ici]

    # update the estimates and the weights (inverse variance weighting)
    weight_theta = 1 / (var_theta + 1e-10)
    yhat = yhat + weight_theta * yhat_theta
    weights = weights + weight_theta

yhat = yhat / weights

Compute the PSNR


In [ ]:
psnr = 10 * np.log10(1 / np.mean((yhat - img) ** 2))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(yhat, cmap="gray")
plt.title(f"LPA-ICI estimate, PSNR = {psnr:.2f}")

Summary comparison figure


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 15))

axes[0, 0].imshow(img, cmap="gray", vmin=0, vmax=1)
axes[0, 0].set_title("Original Image")
axes[0, 0].axis("off")

axes[0, 1].imshow(noisy_img, cmap="gray", vmin=0, vmax=1)
axes[0, 1].set_title(f"Noisy Image (PSNR = {psnr_noisy:.2f} dB)")
axes[0, 1].axis("off")

# Re-run isotropic LPA-ICI for comparison
yhat_iso = np.zeros((img.shape))
best_scale_iso = np.zeros(shape=yhat_iso.shape)
lower_bounds = -np.inf * np.ones(shape=yhat_iso.shape)
upper_bounds = np.inf * np.ones(shape=yhat_iso.shape)

for i, h in enumerate(all_h):
    w = np.zeros((2 * hmax + 1, 2 * hmax + 1))
    w[hmax - h : hmax + h + 1, hmax - h : hmax + h + 1] = 1
    g = compute_2D_LPA_kernel(w, N)

    yhat_h = convolve2d(noisy_img, g, mode="same", boundary="symm")
    var_h = sigma_noise**2 * convolve2d(
        np.ones_like(noisy_img), g**2, mode="same", boundary="symm"
    )

    lb = yhat_h - Gamma * np.sqrt(var_h)
    ub = yhat_h + Gamma * np.sqrt(var_h)

    lower_bounds = np.maximum(lower_bounds, lb)
    upper_bounds = np.minimum(upper_bounds, ub)

    valid_ici = lower_bounds <= upper_bounds
    best_scale_iso[valid_ici] = h
    yhat_iso[valid_ici] = yhat_h[valid_ici]

psnr_iso = 10 * np.log10(1 / np.mean((yhat_iso - img) ** 2))

axes[1, 0].imshow(yhat_iso, cmap="gray", vmin=0, vmax=1)
axes[1, 0].set_title(f"Isotropic LPA-ICI (PSNR = {psnr_iso:.2f} dB)")
axes[1, 0].axis("off")

axes[1, 1].imshow(yhat, cmap="gray", vmin=0, vmax=1)
axes[1, 1].set_title(f"Anisotropic LPA-ICI (PSNR = {psnr:.2f} dB)")
axes[1, 1].axis("off")

plt.tight_layout()
plt.show()

print("PSNR Results:")
print(f"Noisy image: {psnr_noisy:.2f} dB")
print(f"Isotropic LPA-ICI: {psnr_iso:.2f} dB")
print(f"Anisotropic LPA-ICI: {psnr:.2f} dB")
print(f"Improvement (Anisotropic vs Isotropic): {psnr - psnr_iso:.2f} dB")

Implement the LPA-ICI with directional kernels (defined over the quadrants)


In [ ]:
## LPA-ICI with Quadrant-based Directional Kernels

# Set the parameters
N = 1  # maximum degree of polynomial used for fitting
Gamma = 2  # parameter for the confidence intervals in the ICI rule
hmax = 21
all_h = np.arange(1, hmax + 1)

# Define 4 quadrant directions
# 0: Top-right quadrant
# 1: Top-left quadrant
# 2: Bottom-left quadrant
# 3: Bottom-right quadrant
all_quadrants = np.arange(4)

# Generate the LPA kernels for all scales and all quadrant directions
all_g_quad = []

for quad in all_quadrants:
    all_g_quad_direction = []
    for i, h in enumerate(all_h):
        # Define the weights for the scale h and the quadrant direction
        w = np.zeros((2 * hmax + 1, 2 * hmax + 1))

        if quad == 0:  # Top-right quadrant
            for dy in range(-h, 1):  # y from -h to 0
                for dx in range(0, h + 1):  # x from 0 to h
                    if hmax + dy >= 0 and hmax + dx < 2 * hmax + 1:
                        w[hmax + dy, hmax + dx] = 1

        elif quad == 1:  # Top-left quadrant
            for dy in range(-h, 1):  # y from -h to 0
                for dx in range(-h, 1):  # x from -h to 0
                    if hmax + dy >= 0 and hmax + dx >= 0:
                        w[hmax + dy, hmax + dx] = 1

        elif quad == 2:  # Bottom-left quadrant
            for dy in range(0, h + 1):  # y from 0 to h
                for dx in range(-h, 1):  # x from -h to 0
                    if hmax + dy < 2 * hmax + 1 and hmax + dx >= 0:
                        w[hmax + dy, hmax + dx] = 1

        elif quad == 3:  # Bottom-right quadrant
            for dy in range(0, h + 1):  # y from 0 to h
                for dx in range(0, h + 1):  # x from 0 to h
                    if hmax + dy < 2 * hmax + 1 and hmax + dx < 2 * hmax + 1:
                        w[hmax + dy, hmax + dx] = 1

        # Compute and store the kernel g
        g = compute_2D_LPA_kernel(w, N)
        all_g_quad_direction.append(g)

    all_g_quad.append(all_g_quad_direction)

# Initialize variables for quadrant-based LPA-ICI
yhat_quad = np.zeros(img.shape)
weights_quad = np.zeros(img.shape)

# Loop over all quadrant directions
for quad in all_quadrants:
    # Initialize the estimate for the current quadrant direction
    yhat_quad_direction = np.zeros(img.shape)

    # Initialize the matrix of variances for the current quadrant direction
    var_quad_direction = np.zeros(img.shape)

    # Initialize the lower and upper bounds matrices
    lower_bounds = -np.inf * np.ones(img.shape)
    upper_bounds = np.inf * np.ones(img.shape)

    # Loop over all scales for current quadrant
    all_g_quad_direction = all_g_quad[quad]
    for i, h in enumerate(all_h):
        g = all_g_quad_direction[i]

        # Compute the estimate for the scale h
        yhat_h = convolve2d(noisy_img, g, mode="same", boundary="symm")

        # Compute the variance
        var_h = sigma_noise**2 * convolve2d(
            np.ones_like(noisy_img), g**2, mode="same", boundary="symm"
        )

        # Compute the lower and upper bound of the confidence interval for the scale h
        lb = yhat_h - Gamma * np.sqrt(var_h)
        ub = yhat_h + Gamma * np.sqrt(var_h)

        # Update the lower and upper bounds
        lower_bounds = np.maximum(lower_bounds, lb)
        upper_bounds = np.minimum(upper_bounds, ub)

        # Identify valid ICI points
        valid_ici = lower_bounds <= upper_bounds

        # Update the estimate
        yhat_quad_direction[valid_ici] = yhat_h[valid_ici]

        # Update the matrix with the variances
        var_quad_direction[valid_ici] = var_h[valid_ici]

    # Update the estimates and the weights (inverse variance weighting)
    weight_quad_direction = 1 / (var_quad_direction + 1e-10)
    yhat_quad = yhat_quad + weight_quad_direction * yhat_quad_direction
    weights_quad = weights_quad + weight_quad_direction

# Final estimate
yhat_quad = yhat_quad / weights_quad

# Compute the PSNR for quadrant-based method
psnr_quad = 10 * np.log10(1 / np.mean((yhat_quad - img) ** 2))

print(f"Quadrant-based LPA-ICI PSNR: {psnr_quad:.2f} dB")

In [ ]:
# Visualization of results comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Original image
axes[0, 0].imshow(img, cmap="gray", vmin=0, vmax=1)
axes[0, 0].set_title("Original Image")
axes[0, 0].axis("off")

# Noisy image
axes[0, 1].imshow(noisy_img, cmap="gray", vmin=0, vmax=1)
axes[0, 1].set_title(f"Noisy Image\n(PSNR = {psnr_noisy:.2f} dB)")
axes[0, 1].axis("off")

# Isotropic LPA-ICI
axes[0, 2].imshow(yhat_iso, cmap="gray", vmin=0, vmax=1)
axes[0, 2].set_title(f"Isotropic LPA-ICI\n(PSNR = {psnr_iso:.2f} dB)")
axes[0, 2].axis("off")

# Anisotropic LPA-ICI (line-based)
axes[1, 0].imshow(yhat, cmap="gray", vmin=0, vmax=1)
axes[1, 0].set_title(f"Line-based Anisotropic\n(PSNR = {psnr:.2f} dB)")
axes[1, 0].axis("off")

# Quadrant-based LPA-ICI
axes[1, 1].imshow(yhat_quad, cmap="gray", vmin=0, vmax=1)
axes[1, 1].set_title(f"Quadrant-based Anisotropic\n(PSNR = {psnr_quad:.2f} dB)")
axes[1, 1].axis("off")

# Difference image (quadrant vs line-based)
diff_img = np.abs(yhat_quad - yhat)
im = axes[1, 2].imshow(diff_img, cmap="hot")
axes[1, 2].set_title("Absolute Difference\n(Quadrant vs Line-based)")
axes[1, 2].axis("off")
plt.colorbar(im, ax=axes[1, 2], shrink=0.6)

plt.tight_layout()
plt.show()

# Print summary of results
print("\n" + "=" * 50)
print("PERFORMANCE COMPARISON")
print("=" * 50)
print(f"Noisy image:                {psnr_noisy:.2f} dB")
print(f"Isotropic LPA-ICI:          {psnr_iso:.2f} dB")
print(f"Line-based Anisotropic:     {psnr:.2f} dB")
print(f"Quadrant-based Anisotropic: {psnr_quad:.2f} dB")
print("-" * 50)
print("Improvement over isotropic:")
print(f"  Line-based:     {psnr - psnr_iso:.2f} dB")
print(f"  Quadrant-based: {psnr_quad - psnr_iso:.2f} dB")
print(f"Quadrant vs Line difference: {psnr_quad - psnr:.2f} dB")